# Sprawozdanie z laboratorium 2

***Autor: Adam Dąbkowski***

Celem drugiego laboratorium jest zaimplementowanie algorytmu genetycznego z selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną. Dodatkowo należy zbadać wpływ doboru parametrów na przykładzie danego problemu.

## 0. Importowanie niezbędnych bibliotek

In [1]:
import random
import copy
from utils import *

## 1. Tworzenie populacji

In [2]:
def create_population(population_size, individual_size):
    population = []

    for i in range(population_size):
        individual = []
        for j in range(individual_size):
            individual.append(random.randint(0, 1))
        population.append(np.array(individual))

    return np.array(population)

## 2. Zdefiniowanie problemu (funkcji celu)

In [3]:
def problem(x):
    h0 = 200
    v0 = 0
    fuel = x.sum()
    m0 = 200 + fuel

    for i in x:
        m0 -= i * 1
        a = (45/m0) * i - 0.09
        v0 = v0 + a
        h0 = h0 + v0

        if (0 <= h0 < 2) and abs(v0) < 2:
            return 2000 - fuel

        elif h0 < 0:
            return -1000 - fuel

    return - fuel

## 3. Implementacja algorytmu genetycznego

Głównym zadaniem drugiego laboratorium jest implementacja algorytmu genetycznego. W tym celu stworzona została klasa ***GeneticAlgorithmSolver***. Podczas tworzenia obiektu tej klasy istnieje możliwość podania czterech parametrów:
- ***t_max*** - liczba iteracji (domyślna wartość wynosi $1000$)
- ***pc*** - prawdopodobieństwo krzyżowania (domyślna wartość wynosi $0,5$)
- ***pm*** - prawdopodobieństwo mutacji (domyślna wartość wynosi $0,5$)
- ***u*** - liczba osobników w populacji (domyślna wartość wynosi $20$)

Klasa ***GeneticAlgorithmSolver*** zawiera także sześć metod:
- ***get_parameters()*** - metoda ta zwraca wartości parametrów
- ***solve()*** - metoda zawierająca w sobie algorytm genetyczny, wykorzystuje metody ***score()*** , ***find_best()*** , ***selection()*** , ***crossing_and_mutation()*** . W wyniku jej wywołania otrzymujemy najlepszego osobnika i wartość funkcji celu
- ***score()*** - metoda zwracająca wyniki dla osobników w populacji
- ***find_best()*** - metoda zwracająca najlepszego osobnika
- ***selection()*** - metoda, która zadaniem jest wybranie lepszych osobników z większym prawdopodobieństwem
- ***crossing_and_mutation()*** - metoda odpowiedzialna za krzyżowanie i mutację

In [4]:
class GeneticAlgorithmSolver:
    def __init__(self, t_max=1000, pc=0.5, pm=0.005, u=50):
        self._t_max = t_max
        self._pc = pc
        self._pm = pm
        self._u = u

    def get_parameters(self):
        return {
            "t_max": self._t_max,
            "pc": self._pc,
            "pm": self._pm,
            "u": self._u
        }

    def solve(self, problem, pop0):
        t = 0
        P = pop0
        O = self.score(problem, P)
        x_best, o_best = self.find_best(P, O)
        while t < self._t_max:
            S = self.selection(P, O, self._u)
            M = self.crossing_and_mutation(S, self._pc, self._pm, self._u)
            O = self.score(problem, M)
            x, o = self.find_best(M, O)

            if o > o_best:
                x_best = x
                o_best = o

            P = M
            t += 1

        return x_best, o_best

    @staticmethod
    def score(problem, P):
        return [problem(x) for x in P]

    @staticmethod
    def find_best(P, O):
        index = np.argmax(O)
        return P[index], O[index]

    @staticmethod
    def selection(P, O, u):
        O = [o + (1000 + P.shape[1]) for o in O]
        O = [o / sum(O) for o in O]
        indices = np.arange(0, P.shape[0])
        selected_indices = np.random.choice(indices, u, replace=True, p=O)

        return P[selected_indices]

    @staticmethod
    def crossing_and_mutation(S, pc, pm, u):
        M = []

        # crossing
        for i in range(0, u - 1, 2):
            individual_1 = S[i]
            individual_2 = S[i + 1]

            if random.random() < pc:
                crossing_point = random.randint(1, len(individual_1))
                individual_1_crossed = copy.deepcopy(individual_1)
                individual_2_crossed = copy.deepcopy(individual_2)

                individual_1_crossed[crossing_point:] = individual_2[crossing_point:]
                individual_2_crossed[crossing_point:] = individual_1[crossing_point:]

                M.append(individual_1_crossed)
                M.append(individual_2_crossed)

            else:
                M.append(individual_1)
                M.append(individual_2)

        if u % 2 == 1:
            M.append(S[-1])

        # mutation
        for individual in M:
            for i in range(individual.shape[0]):
                if random.random() < pm:
                    individual[i] = 1 if individual[i] == 0 else 0

        return np.array(M)

## 4. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika

Po zaimplementowaniu algorytmu genetycznego, a także po zdefiniowaniu funkcji celu możemy przejść do testowania solvera w celu znalezienia największej wartości wspomnianej funkcji. Na początku wykorzystany zostanie solver z domyślnymi wartościami, jednakże należy najpierw utworzyć populację początkową. W naszych doświadczeniach liczba osobników będzie wynosiła $1000$, natomiast wielkość osobnika będzie równa $200$.

In [5]:
pop0 = create_population(population_size=1000, individual_size=200)

Następnie przechodzimy do stworzenia solvera.

In [6]:
solver = GeneticAlgorithmSolver()

Chcąc upewnić się jakie parametry ma nasz solver możemy wykorzystać metodę ***get_parameters()*** .

In [7]:
solver.get_parameters()

{'t_max': 1000, 'pc': 0.5, 'pm': 0.005, 'u': 50}

Teraz możemy już przejść do rozwiązania naszego problemu.

In [8]:
x_best, o_best = solver.solve(problem, pop0)

In [9]:
print(f'Best score = {o_best}')

Best score = 1926


In [10]:
x_best

array([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0])

## 5. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika - Badanie wpływu hiperparametrów


Aby usprawnić analizę otrzymanych wyników, zaimplementowana została klasa ***GeneticAlgorithmAnalyzer***, która podczas tworzenia obiektu przyjmuje jako argumenty **solver**, **funkcję celu**, **populację początkową** oraz **liczbę przebiegów**, na podstawie których będziemy mogli ocenić dobór parametrów solvera.

Na początku przetestujemy solver z domyślnymi wartościami.

In [11]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [12]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1923
Sample 3: Best score = 1924
Sample 4: Best score = 1924
Sample 5: Best score = 1920
Sample 6: Best score = 1921
Sample 7: Best score = 1922
Sample 8: Best score = 1924
Sample 9: Best score = 1923
Sample 10: Best score = 1925
Sample 11: Best score = 1922
Sample 12: Best score = 1922
Sample 13: Best score = 1924
Sample 14: Best score = 1921
Sample 15: Best score = 1925
Sample 16: Best score = 1922
Sample 17: Best score = 1925
Sample 18: Best score = 1922
Sample 19: Best score = 1925
Sample 20: Best score = 1921
Sample 21: Best score = 1924
Sample 22: Best score = 1922
Sample 23: Best score = 1922
Sample 24: Best score = 1926
Sample 25: Best score = 1925
 
Best sample = 24
Best score = 1926, Minimum score = 1920
Average score = 1923.16, Median score = 1923.0


Jak widać powyżej, wartość minimalna różni się od maksymalnej, natomiast średnia wyników jest zbliżona do ich mediany. Na podstawie otrzymanych wyników możemy stwierdzić, że w każdym przebiegu rakieta wylądowała.

Sprawdźmy jednak, czy uda nam się zredukować koszty startu rakiety tak, aby jeszcze bardziej zbliżyć się do wartości równej $2000$.

#### 5.1 Badanie wpływu prawdopodobieństwa krzyżowania

In [14]:
solver = GeneticAlgorithmSolver(pc=0.7)

In [15]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [16]:
analyzer.analyze()

Sample 1: Best score = 1920
Sample 2: Best score = 1922
Sample 3: Best score = 1925
Sample 4: Best score = 1923
Sample 5: Best score = 1924
Sample 6: Best score = 1924
Sample 7: Best score = 1921
Sample 8: Best score = 1924
Sample 9: Best score = 1927
Sample 10: Best score = 1922
Sample 11: Best score = 1924
Sample 12: Best score = 1924
Sample 13: Best score = 1925
Sample 14: Best score = 1922
Sample 15: Best score = 1923
Sample 16: Best score = 1922
Sample 17: Best score = 1924
Sample 18: Best score = 1925
Sample 19: Best score = 1921
Sample 20: Best score = 1923
Sample 21: Best score = 1921
Sample 22: Best score = 1924
Sample 23: Best score = 1923
Sample 24: Best score = 1922
Sample 25: Best score = 1924
 
Best sample = 9
Best score = 1927, Minimum score = 1920
Average score = 1923.16, Median score = 1923.0


In [17]:
solver = GeneticAlgorithmSolver(pc=0.4)

In [18]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [19]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1922
Sample 3: Best score = 1923
Sample 4: Best score = 1922
Sample 5: Best score = 1921
Sample 6: Best score = 1923
Sample 7: Best score = 1923
Sample 8: Best score = 1922
Sample 9: Best score = 1920
Sample 10: Best score = 1923
Sample 11: Best score = 1922
Sample 12: Best score = 1921
Sample 13: Best score = 1922
Sample 14: Best score = 1924
Sample 15: Best score = 1926
Sample 16: Best score = 1925
Sample 17: Best score = 1922
Sample 18: Best score = 1925
Sample 19: Best score = 1922
Sample 20: Best score = 1920
Sample 21: Best score = 1921
Sample 22: Best score = 1926
Sample 23: Best score = 1921
Sample 24: Best score = 1923
Sample 25: Best score = 1923
 
Best sample = 15
Best score = 1926, Minimum score = 1920
Average score = 1922.64, Median score = 1922.0


In [20]:
solver = GeneticAlgorithmSolver(pc=0.3)

In [21]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [22]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1924
Sample 4: Best score = 1921
Sample 5: Best score = 1922
Sample 6: Best score = 1926
Sample 7: Best score = 1921
Sample 8: Best score = 1922
Sample 9: Best score = 1925
Sample 10: Best score = 1923
Sample 11: Best score = 1919
Sample 12: Best score = 1920
Sample 13: Best score = 1923
Sample 14: Best score = 1922
Sample 15: Best score = 1923
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1923
Sample 19: Best score = 1923
Sample 20: Best score = 1924
Sample 21: Best score = 1923
Sample 22: Best score = 1921
Sample 23: Best score = 1920
Sample 24: Best score = 1921
Sample 25: Best score = 1921
 
Best sample = 6
Best score = 1926, Minimum score = 1919
Average score = 1922.12, Median score = 1922.0


In [23]:
solver = GeneticAlgorithmSolver(pc=0.2)

In [24]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [25]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1923
Sample 3: Best score = 1926
Sample 4: Best score = 1925
Sample 5: Best score = 1920
Sample 6: Best score = 1925
Sample 7: Best score = 1924
Sample 8: Best score = 1921
Sample 9: Best score = 1921
Sample 10: Best score = 1922
Sample 11: Best score = 1924
Sample 12: Best score = 1921
Sample 13: Best score = 1924
Sample 14: Best score = 1921
Sample 15: Best score = 1920
Sample 16: Best score = 1923
Sample 17: Best score = 1922
Sample 18: Best score = 1925
Sample 19: Best score = 1923
Sample 20: Best score = 1927
Sample 21: Best score = 1923
Sample 22: Best score = 1923
Sample 23: Best score = 1921
Sample 24: Best score = 1922
Sample 25: Best score = 1921
 
Best sample = 20
Best score = 1927, Minimum score = 1920
Average score = 1922.84, Median score = 1923.0


In [26]:
solver = GeneticAlgorithmSolver(pc=0.1)

In [27]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [28]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1921
Sample 3: Best score = 1923
Sample 4: Best score = 1923
Sample 5: Best score = 1921
Sample 6: Best score = 1922
Sample 7: Best score = 1922
Sample 8: Best score = 1924
Sample 9: Best score = 1924
Sample 10: Best score = 1926
Sample 11: Best score = 1923
Sample 12: Best score = 1923
Sample 13: Best score = 1925
Sample 14: Best score = 1924
Sample 15: Best score = 1925
Sample 16: Best score = 1921
Sample 17: Best score = 1924
Sample 18: Best score = 1924
Sample 19: Best score = 1923
Sample 20: Best score = 1922
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1921
Sample 24: Best score = 1921
Sample 25: Best score = 1924
 
Best sample = 10
Best score = 1926, Minimum score = 1921
Average score = 1923.16, Median score = 1923.0


In [29]:
solver = GeneticAlgorithmSolver(pc=0.05)

In [30]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [31]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1920
Sample 3: Best score = 1923
Sample 4: Best score = 1921
Sample 5: Best score = 1923
Sample 6: Best score = 1921
Sample 7: Best score = 1922
Sample 8: Best score = 1919
Sample 9: Best score = 1924
Sample 10: Best score = 1920
Sample 11: Best score = 1923
Sample 12: Best score = 1922
Sample 13: Best score = 1921
Sample 14: Best score = 1921
Sample 15: Best score = 1921
Sample 16: Best score = 1924
Sample 17: Best score = 1922
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1921
Sample 21: Best score = 1922
Sample 22: Best score = 1924
Sample 23: Best score = 1923
Sample 24: Best score = 1919
Sample 25: Best score = 1922
 
Best sample = 9
Best score = 1924, Minimum score = 1919
Average score = 1921.76, Median score = 1922.0


In [32]:
solver = GeneticAlgorithmSolver(pc=0.01)

In [33]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [34]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1920
Sample 3: Best score = 1921
Sample 4: Best score = 1922
Sample 5: Best score = 1921
Sample 6: Best score = 1920
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1923
Sample 10: Best score = 1923
Sample 11: Best score = 1923
Sample 12: Best score = 1923
Sample 13: Best score = 1923
Sample 14: Best score = 1921
Sample 15: Best score = 1922
Sample 16: Best score = 1922
Sample 17: Best score = 1922
Sample 18: Best score = 1921
Sample 19: Best score = 1920
Sample 20: Best score = 1923
Sample 21: Best score = 1922
Sample 22: Best score = 1921
Sample 23: Best score = 1920
Sample 24: Best score = 1923
Sample 25: Best score = 1923
 
Best sample = 9
Best score = 1923, Minimum score = 1920
Average score = 1921.8, Median score = 1922.0


In [35]:
solver = GeneticAlgorithmSolver(pc=0.005)

In [36]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [37]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1920
Sample 4: Best score = 1921
Sample 5: Best score = 1925
Sample 6: Best score = 1921
Sample 7: Best score = 1923
Sample 8: Best score = 1921
Sample 9: Best score = 1919
Sample 10: Best score = 1924
Sample 11: Best score = 1920
Sample 12: Best score = 1922
Sample 13: Best score = 1924
Sample 14: Best score = 1921
Sample 15: Best score = 1920
Sample 16: Best score = 1923
Sample 17: Best score = 1923
Sample 18: Best score = 1921
Sample 19: Best score = 1924
Sample 20: Best score = 1919
Sample 21: Best score = 1924
Sample 22: Best score = 1925
Sample 23: Best score = 1922
Sample 24: Best score = 1924
Sample 25: Best score = 1920
 
Best sample = 5
Best score = 1925, Minimum score = 1919
Average score = 1922.0, Median score = 1922.0


In [38]:
solver = GeneticAlgorithmSolver(pc=0.001)

In [39]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, problem=problem, pop0=pop0, n_samples=25)

In [40]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1923
Sample 3: Best score = 1921
Sample 4: Best score = 1926
Sample 5: Best score = 1923
Sample 6: Best score = 1919
Sample 7: Best score = 1920
Sample 8: Best score = 1922
Sample 9: Best score = 1924
Sample 10: Best score = 1922
Sample 11: Best score = 1921
Sample 12: Best score = 1922
Sample 13: Best score = 1925
Sample 14: Best score = 1926
Sample 15: Best score = 1920
Sample 16: Best score = 1924
Sample 17: Best score = 1921
Sample 18: Best score = 1922
Sample 19: Best score = 1919
Sample 20: Best score = 1923
Sample 21: Best score = 1919
Sample 22: Best score = 1921
Sample 23: Best score = 1923
Sample 24: Best score = 1921
Sample 25: Best score = 1921
 
Best sample = 4
Best score = 1926, Minimum score = 1919
Average score = 1922.0, Median score = 1922.0


#### 5.2 Badanie wpływu prawdopodobieństwa mutacji

#### 5.3 Badanie wpływu wielkości populacji

In [13]:
solver = GeneticAlgorithmSolver()